In [1]:
import numpy as np 
import pandas as pd 
import sqlite3
import matplotlib.pyplot as plt


database = "/Users/paulmckee/Desktop/Data Engineering/database.sqlite"

In [2]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [3]:
countries = pd.read_sql("""SELECT *
                        FROM Country;""", conn)
countries

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


What matches were played in English Premier League during 2009/10 season and what was the score from each match?

In [4]:
detailed_matches = pd.read_sql("""
SELECT Match.id, 
Country.name AS country_name, 
League.name AS league_name, 
season, 
date,
HT.team_long_name AS  home_team,
AT.team_long_name AS away_team,
home_team_goal, 
away_team_goal                                        
FROM Match
LEFT JOIN Country on Country.id = Match.country_id
LEFT JOIN League on League.id = Match.league_id
LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
WHERE country_name = 'England'
AND season = '2009/2010'
ORDER by date
LIMIT 10;""", conn)
detailed_matches

,id,country_name,league_name,season,date,home_team,away_team,home_team_goal,away_team_goal
0,2111,England,England Premier League,2009/2010,2009-08-15 00:00:00,Aston Villa,Wigan Athletic,0,2
1,2112,England,England Premier League,2009/2010,2009-08-15 00:00:00,Everton,Arsenal,1,6
2,2113,England,England Premier League,2009/2010,2009-08-15 00:00:00,Wolverhampton Wanderers,West Ham United,0,2
3,2114,England,England Premier League,2009/2010,2009-08-15 00:00:00,Blackburn Rovers,Manchester City,0,2
4,2115,England,England Premier League,2009/2010,2009-08-15 00:00:00,Bolton Wanderers,Sunderland,0,1
5,2116,England,England Premier League,2009/2010,2009-08-15 00:00:00,Portsmouth,Fulham,0,1
6,2117,England,England Premier League,2009/2010,2009-08-15 00:00:00,Stoke City,Burnley,2,0
7,2118,England,England Premier League,2009/2010,2009-08-15 00:00:00,Chelsea,Hull City,2,1
8,2109,England,England Premier League,2009/2010,2009-08-16 00:00:00,Manchester United,Birmingham City,1,0
9,2110,England,England Premier League,2009/2010,2009-08-16 00:00:00,Tottenham Hotspur,Liverpool,2,1


middlesbrough only

In [5]:
tot = pd.read_sql("""
SELECT m.date,
HT.team_long_name AS  home_team,
AT.team_long_name AS away_team,
home_team_goal, 
away_team_goal, 
CASE WHEN m.home_team_api_id = 8549 and m.home_team_goal > m.away_team_goal THEN 'You bet they won'
WHEN m.home_team_api_id = 8549 and  m.home_team_goal < m.away_team_goal THEN 'Sadly they lost'
WHEN m.away_team_api_id = 8549 and m.home_team_goal < m.away_team_goal THEN 'You bet they won'
WHEN m.away_team_api_id = 8549 and m.home_team_goal > m.away_team_goal THEN 'Sadly they lost'
ELSE 'Not good enough to win, not bad enough to lose' END AS outcome
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8549 or m.away_team_api_id = 8549
Order by 1;""", conn)
tot

,date,home_team,away_team,home_team_goal,away_team_goal,outcome
0,2008-08-16 00:00:00,Middlesbrough,Tottenham Hotspur,2,1,You bet they won
1,2008-08-23 00:00:00,Liverpool,Middlesbrough,2,1,Sadly they lost
2,2008-08-30 00:00:00,Middlesbrough,Stoke City,2,1,You bet they won
3,2008-09-13 00:00:00,Portsmouth,Middlesbrough,2,1,Sadly they lost
4,2008-09-20 00:00:00,Sunderland,Middlesbrough,2,0,Sadly they lost
5,2008-09-27 00:00:00,Middlesbrough,West Bromwich Albion,0,1,Sadly they lost
6,2008-10-04 00:00:00,Wigan Athletic,Middlesbrough,0,1,You bet they won
7,2008-10-18 00:00:00,Middlesbrough,Chelsea,0,5,Sadly they lost
8,2008-10-25 00:00:00,Blackburn Rovers,Middlesbrough,1,1,"Not good enough to win, not bad enough to lose"
9,2008-10-29 00:00:00,Middlesbrough,Manchester City,2,0,You bet they won


goals per season for Middlesbrough

In [6]:
GoalsBySeason = pd.read_sql("""
SELECT
m.season,
SUM(CASE WHEN m.home_team_api_id = 8549  THEN home_team_goal END) AS home_goals,
SUM(CASE WHEN m.away_team_api_id = 8549  THEN away_team_goal END) AS away_goals
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8549 or m.away_team_api_id = 8549
GROUP BY m.season;""", conn)
GoalsBySeason

,season,home_goals,away_goals
0,2008/2009,17,11
